### APPLY HMM from scratch

In [66]:
vocab = ['nitish','loves','facebook','can','google','will','ankita']
sentences = [
    'Nitish Loves Facebook',
    'Can Nitish google facebook',
    'Will ankita google facebook',
    'Ankita loves Will',
    'Will loves google'
]

In [34]:
import spacy

In [35]:
# !python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [187]:
# doc1 = sentences[0]
# doc2 = sentences[1]
# doc3 = sentences[2]
# doc4 = sentences[3]
# doc5 = sentences[4]

# docs = [doc1,doc2,doc3,doc4,doc5]
# for doc in docs:
#     x = nlp(doc)
#     for word in x:
#         print(word.text,'-->', word.pos_ ,word.tag_ , spacy.explain(word.tag_))


tagged_sentences = []
for s in sentences:
    doc = nlp(s)
    tag = [(word.text.lower(), word.pos_) for word in doc]
    tagged_sentences.append(tag)

# Modify "Will" in the last two sentences to be a noun and "google" as verb
tagged_sentences[1] = [(word, pos) if word != 'google' else (word, 'VERB') for word, pos in tagged_sentences[1]]
tagged_sentences[2] = [(word, pos) if word != 'google' else (word, 'VERB') for word, pos in tagged_sentences[2]]
tagged_sentences[3] = [(word, pos) if word != 'will' else (word, 'NOUN') for word, pos in tagged_sentences[3]]
tagged_sentences[4] = [(word, pos) if word != 'will' else (word, 'NOUN') for word, pos in tagged_sentences[4]]

for i,s in enumerate(tagged_sentences): 
    tagged_sentences[i] = [(word,'NOUN') if pos == 'PROPN' else (word,pos) for word,pos in s]

markers = []
for tag in tagged_sentences:
    m = [('S', 'START')] + tag + [('E', 'END')]
    markers.append(m)
    
all_tags = {}
for i, tagged in enumerate(markers):
    all_tags[i] = tagged
    print(f"Sentence {i + 1}: {tagged}")

Sentence 1: [('S', 'START'), ('nitish', 'NOUN'), ('loves', 'VERB'), ('facebook', 'NOUN'), ('E', 'END')]
Sentence 2: [('S', 'START'), ('can', 'AUX'), ('nitish', 'NOUN'), ('google', 'VERB'), ('facebook', 'NOUN'), ('E', 'END')]
Sentence 3: [('S', 'START'), ('will', 'AUX'), ('ankita', 'NOUN'), ('google', 'VERB'), ('facebook', 'NOUN'), ('E', 'END')]
Sentence 4: [('S', 'START'), ('ankita', 'NOUN'), ('loves', 'VERB'), ('will', 'NOUN'), ('E', 'END')]
Sentence 5: [('S', 'START'), ('will', 'NOUN'), ('loves', 'VERB'), ('google', 'NOUN'), ('E', 'END')]


In [188]:
all_tags

{0: [('S', 'START'),
  ('nitish', 'NOUN'),
  ('loves', 'VERB'),
  ('facebook', 'NOUN'),
  ('E', 'END')],
 1: [('S', 'START'),
  ('can', 'AUX'),
  ('nitish', 'NOUN'),
  ('google', 'VERB'),
  ('facebook', 'NOUN'),
  ('E', 'END')],
 2: [('S', 'START'),
  ('will', 'AUX'),
  ('ankita', 'NOUN'),
  ('google', 'VERB'),
  ('facebook', 'NOUN'),
  ('E', 'END')],
 3: [('S', 'START'),
  ('ankita', 'NOUN'),
  ('loves', 'VERB'),
  ('will', 'NOUN'),
  ('E', 'END')],
 4: [('S', 'START'),
  ('will', 'NOUN'),
  ('loves', 'VERB'),
  ('google', 'NOUN'),
  ('E', 'END')]}

In [189]:
tag = set()
for tags in all_tags.values():
    for word, pos in tags:
        tag.add(pos)
        
POS_tags = sorted(tag)

print("Unique POS tags:", POS_tags)

Unique POS tags: ['AUX', 'END', 'NOUN', 'START', 'VERB']


In [190]:
from collections import defaultdict

emission_counts = defaultdict(lambda: defaultdict(int))

# Count occurrences
for x in tagged_sentences:
    for word, pos in x:
        emission_counts[pos][word] += 1
        
print("\nEmission Counts:")
for pos, words in emission_counts.items():
    print(f"{pos}: {dict(words)}")


Emission Counts:
NOUN: {'nitish': 2, 'facebook': 3, 'ankita': 2, 'will': 2, 'google': 1}
VERB: {'loves': 3, 'google': 2}
AUX: {'can': 1, 'will': 1}


In [191]:
# emmission probability
probability = defaultdict(lambda: defaultdict(float))

for pos,words in emission_counts.items():
    total = sum(words.values())
    for word,count in words.items():
        probability[pos][word] = count / total

em_probabilities = {pos: dict(words) for pos, words in probability.items()}

print('Probability')
for pos,words in probability.items():
    for word,prob in words.items():
        store = f"P({word}|{pos}) = {prob:.2f}"
        print(store)
        

Probability
P(nitish|NOUN) = 0.20
P(facebook|NOUN) = 0.30
P(ankita|NOUN) = 0.20
P(will|NOUN) = 0.20
P(google|NOUN) = 0.10
P(loves|VERB) = 0.60
P(google|VERB) = 0.40
P(can|AUX) = 0.50
P(will|AUX) = 0.50


In [192]:
em_probabilities

{'NOUN': {'nitish': 0.2,
  'facebook': 0.3,
  'ankita': 0.2,
  'will': 0.2,
  'google': 0.1},
 'VERB': {'loves': 0.6, 'google': 0.4},
 'AUX': {'can': 0.5, 'will': 0.5}}

In [193]:
transition_counts = defaultdict(lambda: defaultdict(int))

for sentence in markers:
    for i in range(len(sentence)):
        current_pos = sentence[i-1][1]
        next_pos = sentence[i][1]
        transition_counts[current_pos][next_pos] += 1


print("\nTransition Counts:")
for current_pos, transitions in transition_counts.items():
    print(f"{current_pos}: {dict(transitions)}") 



Transition Counts:
END: {'START': 5}
START: {'NOUN': 3, 'AUX': 2}
NOUN: {'VERB': 5, 'END': 5}
VERB: {'NOUN': 5}
AUX: {'NOUN': 2}


In [194]:
transition_probabity = defaultdict(lambda: defaultdict(float))

for current_pos,transitions in transition_counts.items():
    total_transitions = sum(transitions.values())
    for next_pos,count in transitions.items():
        transition_probabity[current_pos][next_pos] = count / total_transitions
        
trans_probabilities = {pos: dict(transitions) for pos, transitions in transition_probabity.items()}


print('\nTransition Probability')
for current_pos,transitions in transition_probabity.items():
    for next_pos,prob in transitions.items():
        store = f"P({next_pos}|{current_pos}) = {prob:.2f}"
        print(store)


Transition Probability
P(START|END) = 1.00
P(NOUN|START) = 0.60
P(AUX|START) = 0.40
P(VERB|NOUN) = 0.50
P(END|NOUN) = 0.50
P(NOUN|VERB) = 1.00
P(NOUN|AUX) = 1.00


In [195]:
trans_probabilities

{'END': {'START': 1.0},
 'START': {'NOUN': 0.6, 'AUX': 0.4},
 'NOUN': {'VERB': 0.5, 'END': 0.5},
 'VERB': {'NOUN': 1.0},
 'AUX': {'NOUN': 1.0}}

In [196]:
s1 = "will Will google facebook"

In [197]:
new = s1.split()
new

['will', 'Will', 'google', 'facebook']

In [206]:
import numpy as np

def predict_tags(sentence, states, emission_probs, transition_probs):
    num_states = len(states)
    num_words = len(sentence)
    viterbi = np.zeros((num_words, num_states))
    backpointer = np.zeros((num_words, num_states), dtype=int)

    initial_probs = transition_probs.get('START', {state: 0 for state in states})
    for s, state in enumerate(states):
        viterbi[0, s] = initial_probs.get(state, 0) * emission_probs[state].get(sentence[0], 0)

    for t in range(1, num_words):
        for s, state in enumerate(states):
            max_prob, max_state = max(
                (viterbi[t-1, prev_s] * transition_probs.get(states[prev_s], {}).get(state, 0) * emission_probs[state].get(sentence[t], 0), prev_s)
                for prev_s in range(num_states)
            )
            viterbi[t, s] = max_prob
            backpointer[t, s] = max_state

    # Backtrack to find the best path
    best_path = np.zeros(num_words, dtype=int)
    best_path[-1] = np.argmax(viterbi[-1])

    for t in range(num_words - 2, -1, -1):
        best_path[t] = backpointer[t + 1, best_path[t + 1]]

    best_tags = [states[state] for state in best_path]

    return print(best_tags)

In [207]:
predicted_tags = predict_tags(new, POS_tags, em_probabilities, trans_probabilities)

['VERB', 'VERB', 'VERB', 'AUX']
